In [1]:
import numpy as np
import itertools
import galois
from tqdm import *
# from sympy import Matrix
# from scipy.special import comb, perm

In [2]:
class inversion:
    def __init__(self) -> None:
        self.GF = galois.GF(2) # Galois field F_2
        self.v3_mat = self.enum_vector(3) # a 3 x (2**3) matrix, the columns enumerate all binary vectos in F_2^{3}. Especially, the first column is all zero vector.
        self.dot_result = np.matmul(self.v3_mat.T, self.v3_mat) # dot_result[i, j] = np.dot( self.v3_mat[:, i], self.v3_mat[:, j]), save the result of vectors' inner product to save time
        self.FourTuple = self.gen_Tuple(4) # Choose 4 elements from candidate vectors
        self.TwoTuple = self.gen_Tuple(2) # Choose 2 elements from candidate vectors
        self.TwoTuple_nozero = self.gen_Tuple(2, nozero=True) # Choose 4 elements from candidate vectors without zero vector

    def enum_vector(self, k):
        if k == 1:
            return self.GF([[0,1]])
        else:
            L = self.enum_vector(k-1)
            LL = np.vstack((self.GF(np.zeros((1, L.shape[1]), dtype=np.int8)), L))
            RR = np.vstack((self.GF(np.ones((1, L.shape[1]), dtype=np.int8)), L))
            return np.hstack((LL, RR))
        
    def gen_Tuple(self, length, nozero=False):
        return [ np.array(p) for p in itertools.combinations(range((1 if nozero else 0), self.v3_mat.shape[1]), length)]
        
    
    def reducible(self, Boundary_sets, nH, edges_H, edges_HB, label_H, label_HB):
        for vH_idx in itertools.product(range(1, self.v3_mat.shape[1]), repeat=nH):
            if not np.all([ self.dot_result[ vH_idx[e[0]], vH_idx[e[1]]] == label_H[e_idx]  for e_idx, e in enumerate(edges_H) ]):
                continue

            for boundary_v_idx in itertools.product( *tuple([ range(len(B)) for B in Boundary_sets]) ):
                if np.all( [ self.dot_result[ vH_idx[e[0]], Boundary_sets[e[1]][boundary_v_idx[e[1]]] ] == label_HB[edge_idx] for edge_idx, e in enumerate(edges_HB)] ):
                    return True
        return False
    
    def reduce_K4_minus(self):
        total = 2**5
        for label in tqdm(itertools.product(range(2), repeat=5), total=total):
            label = self.GF(np.array(list(label)))
            if np.all(label[[0,1,2]] == 0):
                continue
            if np.all(label[[0,3,4]] == 0):
                continue
            for boundary_sets_idx in itertools.product(range(len(self.FourTuple)), repeat=2):
                if np.any( self.FourTuple[boundary_sets_idx[0]] == 0 ) and label[1] == 0 and label[3] == 0:
                    continue
                if np.any( self.FourTuple[boundary_sets_idx[1]] == 0 ) and label[2] == 0 and label[4] == 0:
                    continue
                if not self.reducible([ self.FourTuple[i] for i in boundary_sets_idx], 2, [(0,1)], [(0,0), (0,1), (1,0), (1,1)], label[[0]], label[1:]):
                    print('\nFail')
                    return False
        print('\nSucceed')
        return True
    
    def reduce_triangle(self):
        total = 2**3
        for inner_label in tqdm(itertools.product(range(2), repeat=3), total=total):
            inner_label = self.GF(np.array(list(inner_label)))
            for outer_label in itertools.product(range(2), repeat=3):
                outer_label = self.GF(np.array(list(outer_label)))
                if np.any(np.array( [ outer_label[i] == 0 and inner_label[i] == 0 and inner_label[(i-1)%3] == 0 for i in range(3)] )):
                    # inner vertices with all labeled zero edges
                    continue
                for u2_idx in itertools.product(range(1, self.v3_mat.shape[1]), repeat=2):
                    for B3_idx in range(len(self.TwoTuple)):
                        if u2_idx[0] == u2_idx[1] and np.all(self.TwoTuple[B3_idx] != u2_idx[0]):
                            continue
                        if np.any(self.TwoTuple[B3_idx] == 0) and outer_label[2] == 0:
                            continue

                        if not self.reducible([[i] for i in u2_idx] + [self.TwoTuple[B3_idx]], 3, [(0,1), (1,2), (2,0)], [(0,0), (1,1), (2,2)], inner_label, outer_label):
                            print('\nFail')
                            return False
        print('\nSucceed')
        return True

    def reduce_P3(self):
        labels = [self.GF(np.array([1,0,0], dtype=int)), self.GF(np.array([1,1,0], dtype=int)), self.GF(np.array([1,1,1], dtype=int))] 
        for label_idx in range(3):
            label = labels[label_idx]
            total = len(self.TwoTuple) ** 3
            for boundary_sets_idx in tqdm(itertools.product(range(len(self.TwoTuple)), repeat = 3), total=total):
                if np.any(self.TwoTuple[boundary_sets_idx[0]] == 0):
                    continue 
                
                if  np.any(np.array([label[i] == 0 and np.any(self.TwoTuple[boundary_sets_idx[i]] == 0) for i in range(1,3) ])):
                    continue
                
                if not self.reducible([self.TwoTuple[i] for i in boundary_sets_idx], 1, [], [(0,0), (0,1), (0,2)], [], label):
                    print('\nFail')
                    return False
        print('\nSucceed')
        return True
    
    def reducible_K23_speedup(self, boundary_sets):
        for boundary_v_idx in itertools.product( *tuple([ range(len(B)) for B in boundary_sets]) ):
            v0_found = False
            v1_found = False
            for v0_idx in range(1, self.v3_mat.shape[1]):
                if np.all([ self.dot_result[ v0_idx, boundary_sets[i][boundary_v_idx[i]] ] == (1 if i == 0 else 0)  for i in range(3)]):
                    v0_found = True
                    break
            if not v0_found:
                continue

            for v1_idx in range(1, self.v3_mat.shape[1]):
                if np.all([ self.dot_result[ v1_idx, boundary_sets[i][boundary_v_idx[i]] ] == (1 if i == 2 else 0)  for i in range(3)]):
                    v1_found = True
                    break
            if v1_found:
                return True
        return False

    def reduce_K23(self):
        total = (len(self.FourTuple))**3
        for boundary_sets_idx in tqdm(itertools.product(range(len(self.FourTuple)), repeat=3), total=total):
            if not np.any(self.FourTuple[boundary_sets_idx[0]] == 0):
                continue 
            if np.any(self.FourTuple[boundary_sets_idx[1]] == 0):
                continue 
            if not np.any(self.FourTuple[boundary_sets_idx[2]] == 0):
                continue 
            if not self.reducible_K23_speedup([self.FourTuple[i] for i in boundary_sets_idx]):
                print('\nFail')
                return False
        print('\nSucceed')
        return True
    
    def reduce_C4_Type1(self):
        inner_label = self.GF(np.array([1,0,0,0]))
        outer_label = self.GF(np.array([0,0,1,1]))
        total = (self.v3_mat.shape[1])**4
        for boundary_set_idx in tqdm(itertools.product(range(1,self.v3_mat.shape[1]), repeat=4), total=total):
            if not self.reducible([[i] for i in boundary_set_idx], 4, [(0,1), (1,2), (2,3), (3,0)], [(0,0), (1,1), (2,2), (3,3)], inner_label, outer_label):
                print('\nFail')
                return False
        print('\nSucceed')
        return True
    
    def reduce_C4_Type1(self):
        inner_label = self.GF(np.array([1,0,0,0]))
        outer_label = self.GF(np.array([0,0,1,1]))
        total = (self.v3_mat.shape[1]-1)**4
        for boundary_set_idx in tqdm(itertools.product(range(1,self.v3_mat.shape[1]), repeat=4), total=total):
            if not self.reducible([[i] for i in boundary_set_idx], 4, [(0,1), (1,2), (2,3), (3,0)], [(0,0), (1,1), (2,2), (3,3)], inner_label, outer_label):
                print('\nFail')
                return False
        print('\nSucceed')
        return True
    
    def no_two_equal_pairs(self, idx):
        if idx[0] == idx[1] and idx[2] == idx[3]:
            return False
        if idx[0] == idx[2] and idx[1] == idx[3]:
            return False
        if idx[0] == idx[3] and idx[2] == idx[1]:
            return False
        return True
        
    def reduce_C4_Type2(self):
        inner_label = self.GF(np.array([1,0,1,0]))
        outer_label = self.GF(np.array([0,0,0,0]))
        total = (self.v3_mat.shape[1]-1)**4
        for boundary_set_idx in tqdm(itertools.product(range(1,self.v3_mat.shape[1]), repeat=4), total=total):
            if not self.no_two_equal_pairs(boundary_set_idx):
                continue
            if not self.reducible([[i] for i in boundary_set_idx], 4, [(0,1), (1,2), (2,3), (3,0)], [(0,0), (1,1), (2,2), (3,3)], inner_label, outer_label):
                print('\nFail')
                return False
        print('\nSucceed')
        return True
    
    def reduce_bridge(self):
        total = (len(self.TwoTuple_nozero))**4
        for boundary_set_idx in tqdm(itertools.product(range(len(self.TwoTuple_nozero)), repeat=4), total=total):
            if not self.reducible([self.TwoTuple_nozero[i] for i in boundary_set_idx], 2, [(0,1)], [(0,0), (0,1), (1,2), (1,3)], self.GF(np.array([1])), self.GF(np.array([0,0,0,0]))):
                print('\nFail')
                return False
        print('\nSucceed')
        return True

In [3]:
inv = inversion()

In [4]:
inv.reduce_K4_minus()

100%|██████████| 32/32 [04:19<00:00,  8.09s/it]


Succeed


True

In [5]:
inv.reduce_triangle()

100%|██████████| 8/8 [04:08<00:00, 31.01s/it]


Succeed


True

In [6]:
inv.reduce_P3()

100%|██████████| 21952/21952 [00:11<00:00, 1882.16it/s]


Succeed


True

In [7]:
inv.reduce_K23()

100%|██████████| 343000/343000 [04:43<00:00, 1211.94it/s]  


Succeed


True

In [6]:
inv.reduce_C4_Type1()

100%|██████████| 2401/2401 [02:13<00:00, 17.96it/s] 


Succeed


True

In [7]:
inv.reduce_C4_Type2()

100%|██████████| 2401/2401 [02:05<00:00, 19.15it/s] 


Succeed


True

In [4]:
inv.reduce_bridge()

100%|██████████| 194481/194481 [19:26<00:00, 166.74it/s]


Succeed


True